In [ ]:
# 크롤링 속도 향상을 위한 설정
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# headless 설정
chrome_options.headless = True

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
from tqdm import tqdm
from selenium.webdriver.common.by import By
path = './chromedriver' 
num_list = []
df_tq = pd.DataFrame(columns=['date', 'title', 'content'])

n = 0
for i in tqdm(range(1, 29)):
    # 더쿠와 인스티즈는 비회원 검색/접속이 불가능하기 때문에, 구글 검색 결과로 대신 크롤링
    search_url = "https://www.google.com/search?q=site:https://www.instiz.net+%ED%99%98%EC%8A%B9%EC%97%B0%EC%95%A0&tbs=cdr:1,cd_min:7/15/2022,cd_max:11/12/2022&ei=Nd6MY_b7IM-I-Qao2ZVQ&start={}0&sa=N&ved=2ahUKEwi26pHNw-D7AhVPRN4KHahsBQo4-gEQ8tMDegQIHBAV&biw=1920&bih=1057&dpr=1".format(str(i))
    driver = webdriver.Chrome(path)
    driver.get(search_url) 
    time.sleep(3) 
    
    html = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(html, "html.parser")
    title = soup.find_all("h3", class_="LC20lb MBeuO DKV0Md")
    content = soup.find_all("div", class_="VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf")
    for j in range(0, len(title)):
        df_tq.loc[n, 'title'] = title[j].text
        if content[j].text[:4].isdigit():
            df_tq.loc[n, 'date'] = content[j].text[:12]
            df_tq.loc[n, 'content'] = content[j].text[15:]
        else:
            df_tq.loc[n, 'date'] = '2000.00.00'
            df_tq.loc[n, 'content'] = content[j].text
        n += 1

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [03:13<00:00,  6.90s/it]


In [ ]:
# 크롤링 결과
df_tq

,date,title,content
0,2022. 8. 1.,스포주의) 환승연애2 출연진 퇴소 관련 - 인스티즈,포주의 상대 엑스가 퇴소하면서 같이 하차함이건 그나마 말이 되는 퇴소 이유 같다고 ...
1,2022. 10. 16,환승연애 X 커플들 사이의 모든 문제점은 캐스팅 과정에서 ...,시즌 1 때는 보현님이 호민님한테 출연제의 했지만 보현님이 훨씬 더 미련이 많이 ...
2,2022. 9. 5.,"'환승연애2' 출연진, 숙소 위생 상태 논란…""천년의 사랑도 식을 듯""",환승연애2' 출연진이 새로운 논란에 직면했다.기사 전문http://www.topst...
3,2022. 7. 26.,환승연애2 태이 이현 술자리 싸움 - 인스티즈,아 근데 이거 보면서 느낀 게 남성분이 너무 초등학생 같아요 뒤끝 장난 아닌건지 말...
4,2022. 9. 1.,[잡담] 환승연애 사람들 대학 어디여?? - 인스티즈,권 국민대 콘서바토리 라고 다른학교 평생교육원 개념임 모델과 인 거 에타에 올라왔더...
...,...,...,...
274,2022. 11. 4.,(환승연애2) 이때부터 악귀들렸던 희두 - 인스티즈,스크랩 수가 높은 글이에요! 스크랩하기 l 카카오톡 공유. (환승연애2) 이때부터 ...
275,2022. 8. 3.,[잡담] 환승연애2 x 커플 그럼 나연님은 - 인스티즈,"에게 와? 와중에 다 다른학교,, 나 이번에 이현님 찐 퇴소에,, 너무 충격먹어서 ..."
276,2022. 8. 18.,올해 신인 남돌 중에 정승환 얘 진짜 개잘생긴 거 같음 - 인스티즈,OnAir 수석코치님 거의 벤투 영혼의 동반자 수준이네 · 23:01 l 조회 23...
277,2022. 10. 1.,[잡담] 현규해은이거봐 - 인스티즈(instiz) (K)POP/예능 카테고리,"[환승연애2] ""여기 와서 처음으로 행복했어"" 현규-해은 제주도의 밤ㅣ16화 예고...."


In [ ]:
# 결과 저장
df_tq.to_csv('./instiz_data.csv',index = False)